In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO

In [46]:
all_players = []

In [47]:
driver = webdriver.Chrome()

In [48]:
urls = [
    'https://www.eurohockey.com/stats/league/2026/137-elitehockey-ligaen.html?type=1&season=2026&list_number=100&order=7&dir=2&&list_number=0',
    'https://www.eurohockey.com/stats/league/2026/137-elitehockey-ligaen.html?type=1&season=2026&list_number=100&order=7&dir=2&&list_number=100',
    'https://www.eurohockey.com/stats/league/2026/137-elitehockey-ligaen.html?type=1&season=2026&list_number=100&order=7&dir=2&&list_number=200'
]

In [49]:
for url in urls:
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    stats_table = soup.find_all('table')[0]

    rows = stats_table.find_all('tr')[1:]
    player_nations = []

    for row in rows:
        player_cell = row.find('td', class_ = 'player')
        if player_cell:
            flag = player_cell.find('img', class_ = 'flag')
            nation = flag['title'] if flag and 'title' in flag.attrs else ''
            player_nations.append(nation)

    player_nations.append('')

    player_data = pd.read_html(StringIO(str(stats_table)))[0]

    player_data['Nation'] = player_nations

    all_players.append(player_data)

In [50]:
driver.quit()

In [51]:
stats_df = pd.concat(all_players, ignore_index=True)

In [52]:
stats_df = stats_df[~stats_df.apply(lambda row: row.astype(str).str.contains('items', case=True, na=False).any(), axis=1)]

In [54]:
stats_df = stats_df.drop(columns=stats_df.columns[0])
columns_to_drop = stats_df.columns[8:19]
stats_df = stats_df.drop(columns=columns_to_drop)

In [55]:
stats_df.to_csv('stats_w_nations.csv', index=False)